<br>

<div align=center><font color=maroon size=6><b>Custom layers</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 official tutorials: <a href="https://www.tensorflow.org/tutorials" style="text-decoration:none;">TensorFlow Tutorials</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Tutorials > <a href="https://www.tensorflow.org/tutorials/customization/custom_layers" style="text-decoration:none;">Custom layers</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/customization/custom_layers.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

<font size=3 color=maroon>We recommend using `tf.keras` as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.</font>

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.8.0'

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

<br>
<br>

## Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as well as easy ways for you to <font color=maroon size=3>write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full [Keras](https://keras.io) API in the tf.keras package, and the Keras layers are very useful when building your own models.</font>


In [5]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)


# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [6]:
layer

<br>

<font size=3 color=maroon>The full list of pre-existing layers can be seen in [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers).</font> It includes Dense (a fully-connected layer),
Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [7]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

<br>

In [8]:
# Layers have many useful methods. For example, you can inspect 
# all variables in a layer using `layer.variables` 
# and trainable variables using `layer.trainable_variables`. 
# In this case a fully-connected layer will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.18998516, -0.20233265, -0.5169396 , -0.26924112,  0.23680156,
         -0.41681212,  0.23571873, -0.36811084, -0.2878108 ,  0.4451967 ],
        [-0.14571404, -0.16646579, -0.00636947, -0.23897743,  0.5711587 ,
          0.5435062 , -0.07922477,  0.20965767,  0.24693471, -0.4941721 ],
        [-0.04835856,  0.04203743,  0.22867024, -0.14528081,  0.20488882,
          0.28987765, -0.09525532,  0.51188403, -0.35119185,  0.49145967],
        [-0.3596662 ,  0.36716497,  0.05266851, -0.37802055, -0.24389103,
         -0.02778381,  0.41465837, -0.36787635, -0.62820053,  0.15780628],
        [ 0.44898254,  0.28254402, -0.1206758 , -0.07699853, -0.6167374 ,
          0.59098595,  0.6159777 ,  0.1768499 ,  0.2219761 , -0.18772903]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [9]:
layer.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.18998516, -0.20233265, -0.5169396 , -0.26924112,  0.23680156,
         -0.41681212,  0.23571873, -0.36811084, -0.2878108 ,  0.4451967 ],
        [-0.14571404, -0.16646579, -0.00636947, -0.23897743,  0.5711587 ,
          0.5435062 , -0.07922477,  0.20965767,  0.24693471, -0.4941721 ],
        [-0.04835856,  0.04203743,  0.22867024, -0.14528081,  0.20488882,
          0.28987765, -0.09525532,  0.51188403, -0.35119185,  0.49145967],
        [-0.3596662 ,  0.36716497,  0.05266851, -0.37802055, -0.24389103,
         -0.02778381,  0.41465837, -0.36787635, -0.62820053,  0.15780628],
        [ 0.44898254,  0.28254402, -0.1206758 , -0.07699853, -0.6167374 ,
          0.59098595,  0.6159777 ,  0.1768499 ,  0.2219761 , -0.18772903]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

<br>

In [10]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.18998516, -0.20233265, -0.5169396 , -0.26924112,  0.23680156,
         -0.41681212,  0.23571873, -0.36811084, -0.2878108 ,  0.4451967 ],
        [-0.14571404, -0.16646579, -0.00636947, -0.23897743,  0.5711587 ,
          0.5435062 , -0.07922477,  0.20965767,  0.24693471, -0.4941721 ],
        [-0.04835856,  0.04203743,  0.22867024, -0.14528081,  0.20488882,
          0.28987765, -0.09525532,  0.51188403, -0.35119185,  0.49145967],
        [-0.3596662 ,  0.36716497,  0.05266851, -0.37802055, -0.24389103,
         -0.02778381,  0.41465837, -0.36787635, -0.62820053,  0.15780628],
        [ 0.44898254,  0.28254402, -0.1206758 , -0.07699853, -0.6167374 ,
          0.59098595,  0.6159777 ,  0.1768499 ,  0.2219761 , -0.18772903]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

<br>
<br>
<br>

## Implementing custom layers
The best way to implement your own layer is extending the **`tf.keras.Layer`**  `(这里应该是指 tf.keras.layers.Layer)` class and implementing:

1. `__init__` , where you can do all input-independent initialization
2. `build`, where you know the shapes of the input tensors and can do the rest of the initialization
3. `call`, where you do the forward computation
<br>
<br>

<font size=3 color=marooon>**Note that** 
* you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. 
<br>
* However, the advantage of creating them in `build` is that it enables late variable creation based on the shape of the inputs the layer will operate on. 
<br>

* On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.</font>

In [11]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_weight('kernel',
                                      shape=[int(input_shape[-1]), self.num_outputs])
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [12]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [13]:
_

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [14]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


<br>

<font size=3 color=maroon>Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in `tf.keras.layers`, consider filing a [github issue](http://github.com/tensorflow/tensorflow/issues/new) or, even better, sending us a pull request!</font>

<br>
<br>
<br>

## Models: Composing layers

<font size=3 color=maroon>Many interesting layer-like things in machine learning models are implemented by composing existing layers.</font> For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

<font size=3 color=maroon>Typically you inherit from `keras.Model` when you need the model methods like: `Model.fit`,`Model.evaluate`, and `Model.save` (see [Custom Keras layers and models](https://www.tensorflow.org/guide/keras/custom_layers_and_models) for details).

One other feature provided by `keras.Model` (instead of `keras.layers.Layer`) is that in addition to tracking variables, a `keras.Model` also tracks its internal layers, making them easier to inspect.</font>

For example here is a ResNet block:

In [15]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()
    
    
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        x += input_tensor
        
        return tf.nn.relu(x)

In [16]:
block = ResnetIdentityBlock(1, [1, 2, 3])

In [17]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [18]:
block.layers

In [19]:
len(block.variables)

18

In [20]:
block.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  4         
                                                                 
 batch_normalization (BatchN  multiple                 4         
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_1 (Batc  multiple                 8         
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           multiple                  9         
                                                                 
 batch_normalization_2 (Batc  multiple                 12        


<br>

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using `tf.keras.Sequential`:

In [21]:
# help(tf.keras.layers.Conv2D)

In [22]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1), input_shape=(None, None, 3)),
                              tf.keras.layers.BatchNormalization(),
                              
                              tf.keras.layers.Conv2D(2, 1, padding='same'),
                              tf.keras.layers.BatchNormalization(),
                              
                              tf.keras.layers.Conv2D(3, (1, 1)),
                              tf.keras.layers.BatchNormalization()])

my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [23]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, None, None, 1)     4         
                                                                 
 batch_normalization_3 (Batc  (None, None, None, 1)    4         
 hNormalization)                                                 
                                                                 
 conv2d_4 (Conv2D)           (None, None, None, 2)     4         
                                                                 
 batch_normalization_4 (Batc  (None, None, None, 2)    8         
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, None, None, 3)     9         
                                                                 
 batch_normalization_5 (Batc  (None, None, None, 3)    1

<br>
<br>
<br>

# Next steps

Now you can go back to the previous notebook and adapt the linear regression example to use layers and models to be better structured.

<br>
<br>
<br>

```python
# MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.
```

<br>
<br>
<br>